# 🔍 Validation Complète avec DeepChecks - Crime Dataset 2020

## 📋 Compatible scikit-learn 1.8.0

### Table des Matières
1. [Setup Compatibilité](#setup)
2. [Niveau 1 : Intégrité des Données](#niveau-1)
3. [Niveau 2 : Drift et Distribution](#niveau-2)
4. [Niveau 3 : Performance du Modèle](#niveau-3)
5. [Résumé et Recommandations](#resume)

---

## 📦 Installation des Dépendances

In [ ]:
# Installation (si nécessaire)
# !pip install deepchecks pandas scikit-learn joblib matplotlib seaborn scipy

## <a id="setup"></a>🔧 Setup Compatibilité scikit-learn 1.8.0

**⚠️ IMPORTANT : Exécutez cette cellule EN PREMIER avant tous les imports !**

In [ ]:
# ==========================================
# 🔧 FIX COMPATIBILITÉ SCIKIT-LEARN 1.8.0
# ==========================================

import sklearn.metrics
from sklearn.metrics import make_scorer
import numpy as np

def max_error_replacement(y_true, y_pred):
    """Remplace max_error supprimé dans sklearn 1.8.0"""
    return np.max(np.abs(y_true - y_pred))

try:
    if 'max_error' not in sklearn.metrics.get_scorer_names():
        sklearn.metrics._SCORERS['max_error'] = make_scorer(
            max_error_replacement, greater_is_better=False
        )
        print("✅ Fix sklearn 1.8.0 appliqué avec succès")
    else:
        print("✅ Scorer max_error déjà disponible")
except Exception as e:
    print(f"⚠️ Erreur: {e}")

print(f"\n📌 scikit-learn version: {sklearn.__version__}")
print("\n✅ Setup terminé - Continuez !")

## 📚 Imports

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from datetime import datetime

from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation
from deepchecks.tabular.checks import (
    MixedNulls, DataDuplicates, OutlierSampleDetection,
    FeatureFeatureCorrelation, FeatureLabelCorrelation,
    TrainTestFeatureDrift, TrainTestLabelDrift
)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, f1_score

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ks_2samp

warnings.filterwarnings('ignore')
print("✅ Imports réussis")

## 🗂️ Fonction de Mapping des Crimes

In [ ]:
def map_crime_group_4(desc):
    """Regroupe les crimes en 4 catégories principales"""
    if pd.isna(desc):
        return "Other / Fraud / Public Order Crime"
    desc = str(desc).upper()
    
    if any(k in desc for k in ["ASSAULT", "BATTERY", "ROBBERY", "HOMICIDE", "RAPE", "SEX"]):
        return "Violent Crime"
    if any(k in desc for k in ["THEFT", "BURGLARY", "SHOPLIFTING", "VANDALISM"]):
        return "Property & Theft Crime"
    if any(k in desc for k in ["VEHICLE", "DWOC", "MOTOR VEHICLE"]):
        return "Vehicle-Related Crime"
    return "Other / Fraud / Public Order Crime"

print("✅ Fonction de mapping définie")

## 📂 Chargement des Données

In [ ]:
df = pd.read_csv('data/processed/crime_2020_processed.csv')
df['Crime_Group'] = df['Crm Cd Desc'].apply(map_crime_group_4)
print(f"✅ Données chargées : {len(df):,} lignes")
print(f"Shape : {df.shape}")

## 🔧 Préparation des Données

In [ ]:
feature_cols = ['Hour', 'Day_of_week', 'Month_num', 'LAT', 'LON', 'Vict Age', 'AREA']
X = df[feature_cols].copy()
y = df['Crime_Group']

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"Features : {feature_cols}")
print(f"\nClasses :")
for i, name in enumerate(le.classes_):
    count = (y == name).sum()
    print(f"  {i}. {name}: {count:,} ({count/len(y)*100:.1f}%)")

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train), columns=feature_cols, index=X_train.index
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test), columns=feature_cols, index=X_test.index
)

print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")
print(f"NaN train: {X_train_imputed.isna().sum().sum()}")
print(f"NaN test: {X_test_imputed.isna().sum().sum()}")

## 📦 Chargement du Modèle

In [ ]:
try:
    model = joblib.load('models/lightgbm_baseline.joblib')
    print(f"✅ Modèle chargé: {type(model).__name__}")
except:
    model = None
    print("⚠️ Modèle non trouvé (Niveau 3 sera skippé)")

## 🗂️ Création des Datasets DeepChecks

In [ ]:
train_dataset = Dataset(X_train_imputed, label=y_train, features=feature_cols)
test_dataset = Dataset(X_test_imputed, label=y_test, features=feature_cols)
print(f"✅ Datasets créés")
print(f"  Train: {len(train_dataset)} échantillons")
print(f"  Test: {len(test_dataset)} échantillons")

---
# <a id="niveau-1"></a>📊 Niveau 1 : Intégrité des Données
---

In [ ]:
print("="*80)
print("🔍 NIVEAU 1 : INTÉGRITÉ DES DONNÉES")
print("="*80)

try:
    integrity_suite = data_integrity()
    integrity_result = integrity_suite.run(train_dataset)
    integrity_result.show()
    print("\n✅ Suite d'intégrité OK")
except Exception as e:
    print(f"⚠️ Erreur: {e}")

In [ ]:
checks = [
    (MixedNulls(), "Valeurs manquantes"),
    (DataDuplicates(), "Duplicatas"),
    (OutlierSampleDetection(), "Outliers"),
    (FeatureFeatureCorrelation(), "Corrélations FF"),
    (FeatureLabelCorrelation(), "Corrélations FL")
]

for check, name in checks:
    try:
        check.run(train_dataset).show()
        print(f"✅ {name}")
    except Exception as e:
        print(f"⚠️ {name}: {e}")

---
# <a id="niveau-2"></a>📈 Niveau 2 : Drift et Distribution
---

In [ ]:
print("="*80)
print("📈 NIVEAU 2 : DRIFT ET DISTRIBUTION")
print("="*80)

try:
    tt_suite = train_test_validation()
    tt_result = tt_suite.run(train_dataset, test_dataset)
    tt_result.show()
    print("\n✅ Suite train/test OK")
except Exception as e:
    print(f"⚠️ Erreur: {e}")

In [ ]:
print("\n" + "="*80)
print("📊 ANALYSE STATISTIQUE DU DRIFT (Kolmogorov-Smirnov)")
print("="*80)

drift_results = []
for feature in feature_cols:
    stat, pval = ks_2samp(X_train_imputed[feature], X_test_imputed[feature])
    drift_results.append({
        'Feature': feature,
        'KS Statistic': f"{stat:.4f}",
        'P-Value': f"{pval:.4f}",
        'Drift': '🔴 OUI' if pval < 0.05 else '✅ NON'
    })

df_drift = pd.DataFrame(drift_results)
print("\n" + df_drift.to_string(index=False))

n_drift = df_drift['Drift'].str.contains('OUI').sum()
print(f"\n🔍 Résultat : {n_drift}/{len(feature_cols)} features avec drift")

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, col in enumerate(feature_cols):
    axes[i].hist(X_train_imputed[col], bins=30, alpha=0.5, label='Train', density=True)
    axes[i].hist(X_test_imputed[col], bins=30, alpha=0.5, label='Test', density=True)
    axes[i].set_title(col)
    axes[i].legend()

for i in range(len(feature_cols), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('reports/distributions.png', dpi=150, bbox_inches='tight')
print("✅ Graphique sauvegardé: reports/distributions.png")
plt.show()

---
# <a id="niveau-3"></a>🎯 Niveau 3 : Performance du Modèle
---

In [ ]:
print("="*80)
print("🎯 NIVEAU 3 : PERFORMANCE DU MODÈLE")
print("="*80)

if model:
    try:
        eval_suite = model_evaluation()
        eval_result = eval_suite.run(train_dataset, test_dataset, model)
        eval_result.show()
        print("\n✅ Suite d'évaluation OK")
    except Exception as e:
        print(f"⚠️ Erreur: {e}")
else:
    print("⚠️ Pas de modèle chargé - Niveau 3 skippé")

In [ ]:
if model:
    y_pred_train = model.predict(X_train_imputed)
    y_pred_test = model.predict(X_test_imputed)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test, average='weighted')
    gap = train_acc - test_acc
    
    print("\n" + "="*80)
    print("📊 MÉTRIQUES DÉTAILLÉES")
    print("="*80)
    print(f"\nTrain Accuracy : {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"Test Accuracy  : {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"Test F1-Score  : {test_f1:.4f}")
    print(f"Gap Train-Test : {gap:.4f} ({gap*100:.2f}%)")
    
    if gap > 0.1:
        print("\n⚠️ OVERFITTING détecté !")
    else:
        print("\n✅ Bon équilibre train/test")
    
    print("\n" + classification_report(y_test, y_pred_test, target_names=le.classes_))

In [ ]:
if model and hasattr(model, 'feature_importances_'):
    imp_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n" + "="*80)
    print("🌟 FEATURE IMPORTANCE")
    print("="*80)
    print(imp_df.to_string(index=False))
    
    plt.figure(figsize=(10, 6))
    plt.barh(imp_df['Feature'], imp_df['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance - LightGBM')
    plt.tight_layout()
    plt.savefig('reports/feature_importance.png', dpi=150, bbox_inches='tight')
    print("\n✅ Graphique sauvegardé: reports/feature_importance.png")
    plt.show()

---
# <a id="resume"></a>📋 Résumé et Recommandations
---

In [ ]:
print("="*80)
print("📋 RÉSUMÉ GÉNÉRAL - VALIDATION DEEPCHECKS")
print("="*80)
print(f"\n📅 Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Dataset : crime_2020_processed.csv")
print(f"🤖 Modèle : {type(model).__name__ if model else 'N/A'}")

print("\n" + "="*80)
print("✅ NIVEAU 1 : INTÉGRITÉ")
print("="*80)
print(f"  Échantillons    : {len(X_train_imputed):,}")
print(f"  Features        : {len(feature_cols)}")
print(f"  Classes         : {len(le.classes_)}")
print(f"  NaN             : {X_train_imputed.isna().sum().sum()}")
print(f"  Duplicatas      : {X_train_imputed.duplicated().sum()}")

print("\n" + "="*80)
print("✅ NIVEAU 2 : DRIFT")
print("="*80)
if 'n_drift' in locals():
    print(f"  Features drift  : {n_drift}/{len(feature_cols)}")
    if n_drift > 0:
        print("  ⚠️ Drift détecté sur certaines features")
    else:
        print("  ✅ Pas de drift significatif")

print("\n" + "="*80)
print("✅ NIVEAU 3 : PERFORMANCE")
print("="*80)
if model:
    print(f"  Test Accuracy   : {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"  Test F1-Score   : {test_f1:.4f}")
    print(f"  Gap Train-Test  : {gap:.4f}")
else:
    print("  ⚠️ Aucun modèle évalué")

print("\n" + "="*80)
print("💡 RECOMMANDATIONS")
print("="*80)
print("  1. ✅ Données prêtes pour production")
print("  2. 🔄 Surveiller le drift en continu")
print("  3. 📈 Améliorer features si accuracy < 60%")
print("  4. 🚀 Déployer l'API de prédiction")
print("\n✅ Validation terminée !")

## 💾 Sauvegarder les Rapports HTML

In [ ]:
import os
os.makedirs('reports', exist_ok=True)

print("💾 Sauvegarde des rapports HTML...\n")

if 'integrity_result' in locals():
    integrity_result.save_as_html('reports/deepchecks_integrity.html')
    print("  ✅ reports/deepchecks_integrity.html")

if 'tt_result' in locals():
    tt_result.save_as_html('reports/deepchecks_train_test.html')
    print("  ✅ reports/deepchecks_train_test.html")

if model and 'eval_result' in locals():
    eval_result.save_as_html('reports/deepchecks_model_eval.html')
    print("  ✅ reports/deepchecks_model_eval.html")

print("\n✅ Tous les rapports sont sauvegardés !")